In [1]:
import numpy
import sys
from tqdm import tqdm
import re
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

Using TensorFlow backend.


Данные - выборка из 1млн русских анекдотов

In [2]:
raw_text = open("/srv/kkotochigov/JokeGenerator/data/jokes").read()

Переводим в нижний регистр, оставляем русские буквы, цифры и некоторую пунктуацию. Так как данные замусоренные, это сокращает словарь с 200 до до 50 символов.

In [ ]:
raw_text = raw_text.replace("\\\n"," ").replace("\n###","\n").lower().replace('"','').replace('\\','')
raw_text = re.sub('[^\-0-9_а-я:,.\\!\\?\\n]', ' ', raw_text).replace('  ',' ').replace('   ',' ').replace('    ',' ')

Составляем словарь chars и делаем два конвертора, чтобы потом удобнее было работать

In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)

seq_length = 100
dataX=[]
dataY=[]

Нарезаем текст на кейсы скользящим окном (размером seq_length)

Пример такого кейса [125,0,7,15,2] => 250

In [4]:
for line in tqdm(raw_text.split("\n"), total=len(raw_text.split("\n")), unit=" strings"):
    for i in range(0,len(line) - seq_length):
        seq_in = line[i:i+seq_length]
        seq_out = line[i+seq_length]
        dataX.append([char_to_int[c] for c in seq_in])
        dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total patterns:" + str(n_patterns))

100%|██████████| 192591/192591 [01:57<00:00, 1636.56 strings/s]

Total patterns:11233275


Нормализуем все закодированные символы

In [5]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)

Создаем сеть. Изначальная Архитектура - два LSTM слоя (256) и один Dense(256 -> 1)

In [50]:
model = Sequential()
model.add(LSTM(256,input_shape=(X.shape[1],X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [38]:
model = Sequential()
model.add(LSTM(256,input_shape=(X.shape[1],X.shape[2]), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(16, activation="relu"))
model.add(Dense(y.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")

Для тестирования нужен сэмпл, иначе на 19 млн кейсов долго будут считаться

In [ ]:
sample_size = 1000000
import random
import pandas
import datetime
sample_indices = random.sample(range(X.shape[0]),sample_size)
X_sample = X[sample_indices, :]
y_sample = y[sample_indices]

Epoch = 0, TS = 2019-05-07 16:15:50.615884
Epoch = 1, TS = 2019-05-07 16:20:44.798754
Epoch = 2, TS = 2019-05-07 16:25:39.512103
Epoch = 3, TS = 2019-05-07 16:30:34.500128
Epoch = 4, TS = 2019-05-07 16:35:29.733254
Epoch = 5, TS = 2019-05-07 16:40:25.280251


Создадим несколько обработчиков.
- CSVLogger - чтобы после закрытия было видно, что он делает
- lambdaCallback - с выводом времени старта
- modelCheckpoint - чтобы потом можно было подгрузить модель, если kernel упадет

In [ ]:
# Define Callbacks
from keras.callbacks import LambdaCallback, CSVLogger, ModelCheckpoint
lambdaCallback = LambdaCallback(on_epoch_begin = lambda epoch, logs: print("Epoch = {}, TS = {}".format(epoch, datetime.datetime.now())))
csv_logger = CSVLogger('/srv/aprosvetov/training.log')
model_checkpoint = ModelCheckpoint('/srv/aprosvetov/model.h5')

Так как эпохи выводятся в лог и на диск, ставлю verbose = 0. Если оставить verbose = 1, браузер начинает виснуть.
С batch_size больше 4000 не работает.

In [ ]:
model.fit(X_sample, y_sample, epochs=50, batch_size=1024, verbose=0, callbacks = [lambdaCallback, csv_logger, model_checkpoint])

In [ ]:
# model.fit(X, y, epochs=50, batch_size=1024)

Попробуем генерацию текста. Берем случайное начало текста, генерируем наиболее вероятный символ и добавляем его в результат (seq_in).

Начальный текст должен быть ровно 100 символов маленькиими буквами.

In [ ]:
pattern = [char_to_int[x] for x in "Приходит Чукча к жене и говорит: - Как же у тебя тут Вальдшнепы снуют? - Да не боись ты, они же ручные. ".lower()[0:100]]

Генерируемый символ выбираем не жадно, как в оригинале, а из двух наиболее веротяных

In [49]:
def weighted_choice(choices):
   total = sum(w for c, w in choices)
   r = random.uniform(0, total)
   upto = 0
   for c, w in choices:
      if upto + w >= r:
         return c
      upto += w
   assert False, "Shouldn't get here"

for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    # index = numpy.argmax(prediction)
    choices = sorted(list(zip(range(0,len(prediction[0])), prediction[0])), key=lambda x: -x[1])[0:2]
    index = weighted_choice(choices)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

и вольши перер воль на проатили пои сее вое парера пои воль по соазт сооала па серали по пооела в сооалил на соооа - пене пенел в серел поинала поиноо в сооететь в пои воео парера в пено сее волоа - по пооа по пооала в серела па серел пои соа по пооативить на принали вольше пооелои но сееет сооативить на поаза пакооалина па поа по поава но сее волооо в соидали воль поооа - не памооооо пооали волоа - нетела волоа по пенел парало волооои пооенала. - но вое вол вы вы воль пе сееет соаза на сооали выло соа пак вы выло серел в сеен серел в сооет в сел сооалили пак серем соазали по соазал по пеноа - пено выл волоо по пооеоала волои пак вылала воль на прооооить соаза волоои пак вы сее воео по перер в пооенали по сооала волои на права в сооалили по по

KeyboardInterrupt: 